In [1]:
import cv2
import tensorflow
import keras
import scipy as sp
from keras.models import load_model
from keras.models import Model
from keras.applications import vgg16
from keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
from keras.models import *
from keras.layers import *
from keras import optimizers
from keras.utils import np_utils
from scipy.spatial import distance as dist
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
import glob

# from PIL import Image
import requests
from io import BytesIO
import os
import pickle
# import tqdm
import itertools
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

Using TensorFlow backend.


In [2]:
def segment_img(img,stride,npw,nph):
#     img = cv2.imread(img)
    height,width, color = img.shape
    test = 0
    # print(height,width,color)
#     orig = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    orig=img
    num_strides_height = height/stride
    num_strides_width = width/stride
    rows = int(num_strides_height)
    columns = int(num_strides_width)
    blank_image = np.zeros((height,width), np.uint8)
    # print(rows,columns)
    for r in range(rows):
        for c in range(columns):
            x1 = (c)*stride
            y1 = (r)*stride
            x2 = x1+npw
            y2 = y1+nph
            if test == 0:
                in_img = img[y1:y2,x1:x2]
                if(len(in_img.shape)<3):
#                     in_img = in_img.reshape(1,227,227,1)
                    in_img = in_img/255
                else:
                    in_img = in_img.astype('uint8')
#                     in_img = cv2.cvtColor(in_img,cv2.COLOR_RGB2GRAY)
#                     in_img = in_img.reshape(1,227,227,1)
                    in_img = in_img/255
#The below line is used to predict
                pred = model.predict(in_img[np.newaxis,:,:,:])
                pred_class = np.argmax(pred)
#                 result = model.predict(in_img, batch_size=None, verbose=0, steps=None)
#                 re = result[0]
                
                if pred_class == 1:
                    alpha = 0.25
                    print('cracked')
#                     print('confidence: ',re[0]*100,'%')
#                     orig = cv2.rectangle(orig,(x1,y1),(x2,y2),(0,0,255),2)
                    out,le = cracklength(in_img)
#                     out = plot_activation(in_img)
#                     out = out*255
#                     out = plt.imsave("crack.png",out)
#                     out = cv2.imread('crack.png')
#                     out = cv2.applyColorMap(out, cv2.COLORMAP_JET)
#                     out.shape
#                     orig[y1:y2,x1:x2]=out
                    cv2.addWeighted(out, alpha, orig[y1:y2,x1:x2], 1 - alpha,0, orig[y1:y2,x1:x2])
#                     cv2.applyColorMap(out,orig[y1:y2,x1:x2],COLORMAP_JET)
#                     blank_image[y1:y2,x1:x2] = out
#                     orig[y1:y2,x1:x2]=out
#                     orig=orig
                else:
                    jkl=0
                    # print('no crack')
            if ((width-x2 >= npw) or (width==npw)):
                test = 0
            else:
                break
        if ((height-y2) >= nph or (height==nph)):
            test=0
        else:
            break
#     orig = blank_image
    
    return(orig)

In [1]:
def plot_activation(img):
    pred = model.predict(img[np.newaxis,:,:,:])
    pred_class = np.argmax(pred)
  
    weights = model.layers[-1].get_weights()[0]
    class_weights = weights[:, pred_class]
    intermediate = Model(inputs=model.input, outputs=model.get_layer("block5_conv3").output)
    conv_output = intermediate.predict(img[np.newaxis,:,:,:])
    conv_output = np.squeeze(conv_output)
  
    h = int(img.shape[0]/conv_output.shape[0])
    w = int(img.shape[1]/conv_output.shape[1])
  
    activation_maps = sp.ndimage.zoom(conv_output, (h, w, 1), order=1)
    out = np.dot(activation_maps.reshape((img.shape[0]*img.shape[1], 512)), class_weights).reshape(img.shape[0],img.shape[1])
#     plt.imshow(img.astype('float32').reshape(img.shape[0],img.shape[1],3))
#     out = plt.imshow(out, cmap='jet', alpha=0.35)
#     plt.title('Crack' if pred_class == 1 else 'No Crack')
    return(out)

In [ ]:
def cracklength(img):
    pix_width=1
    # img=cv2.imread(img)
    if (len(img)<3):
        img = img
    else:
        # img=img.astype('unit8')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(img, (7, 7), 0)
    img = cv2.Canny(gray, 50, 100)
    cv2.imwrite('Blur & Canny_2.png',img)
    img = cv2.dilate(img, None, iterations=1)
    cv2.imwrite('dialate_3.png',img)
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=4)
    sizes = stats[1:, -1]; nb_components = nb_components - 1
    min_size = 800
    img2 = np.zeros((output.shape))
    for i in range(0, nb_components):
        if sizes[i] >= min_size:
            img2[output == i + 1] = 255
    cv2.imwrite('test.png',img2)
    cv2.imwrite('template_4.png',img2)
    img = cv2.erode(img2, None, iterations=1)
    #read the image with only crack
    img2 = cv2.imread('C:/Users/deepa/Downloads/test.png')
    img3 = cv2.bitwise_and(img2,orig_p)
    img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)
    img3 = cv2.dilate(img3, None, iterations=3)
    ret,thresh = cv2.threshold(img3,127,255,0)
    _,contours,hierarchy = cv2.findContours(thresh, 1, 2)
    n=len(contours)-1
    cnt = contours[n]
    (x,y),radius = cv2.minEnclosingCircle(cnt)
    center = (int(x),int(y))
    radius = int(radius)
    cv2.circle(orig_p,center,radius,(255,255,255),2)
    crack_length = radius*2*pix_width
    cv2.imwrite('circle_5.png',orig_p)
    print('Length of crack is ',crack_length,' centimeters')
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(orig_p,str(crack_length)+" pxls",(150,150),font,0.5,(0,255,255),2,cv2.LINE_AA)
    # cv2.imshow("length",orig_p)
    cv2.imwrite('Length_6.png',orig_p)
    return orig_p,crack_length

In [4]:
model = load_model('D:/SFU/Capstone/CD_V5.h5')
cap = cv2.VideoCapture(0)
stride = 120 #pixels to stride frwd or down
#Shape of neural net dectectable image width x height
neural_pixels_width = 224
neural_pixels_height = 224
npw = neural_pixels_width
nph = neural_pixels_height
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    results = segment_img(frame,stride,npw,nph)
    # Display the resulting frame
    cv2.imshow('frame',results)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
cracked
